# PLN I

## Desafio 3

Se utilizará de base para realizar el desafío la notebook planteada en clase.

Importamos las bilbiotecas que se utilizarán en todo el desafío

In [1]:
import re, math, random
import numpy as np
from pypdf import PdfReader
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import pad_sequences
import matplotlib.pyplot as plt

TensorFlow version: 2.18.1


Luego, setemos una seed = 42 para reproducibilidad de los experimentos que se realizarán a lo largo de todo el desafío

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

### Paso 1: Obtención del corpus, limpieza y tokenización

Definimos una function para obtener el corpus y otra para limpiar de caracteres indeseados

In [3]:
def read_pdfs(folder):
    corpus = []
    pdf_paths = sorted(Path(folder).glob("*.pdf"))
    assert pdf_paths, f"No se encontraron PDFs en la carpeta: {folder}"
    print('PDFs encontrados:')
    for path in pdf_paths:
        print("  •", path.name)
        reader = PdfReader(str(path))
        pages_text = [page.extract_text() or '' for page in reader.pages]
        corpus.append('\n'.join(pages_text))
    return '\n'.join(corpus).lower()

In [4]:
def clean_text(text: str) -> str:
    text = re.sub(r"[^a-záéíóúñü0-9,.;:\s\-\'\"!?()\n]", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

Luego, obtenemos el texto de los pdf y limpiamos

In [5]:
raw_text = read_pdfs(r"./textos")
text = clean_text(raw_text)

PDFs encontrados:
  • Fiodor Mijailovich Dostoyevski - Crimen y Castigo.pdf
  • Franz Kafka - La Metamorfosis.pdf
  • Jane Austen - Orgullo y Prejuicio.pdf
  • Julio Verne - La Vuelta al Mundo en 80 dias.pdf
  • Oscar Wilde - El Retrato de Dorian Gray.pdf


A continuación, obtenemos el vocabulario y tokenizamos

In [6]:
chars = sorted(set(text))
vocab_size = len(chars)
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for c,i in char2idx.items()}
encoded = np.array([char2idx[c] for c in text], dtype=np.int16)
print('Tamaño del vocabulario:', vocab_size)

Tamaño del vocabulario: 55


Se obtiene un tamaño de vocabulario de 55 elementos

### Paso 2: preparación del dataset y functions para el entrenamiento

Dividimos el set en 90% entrenamiento y 10% validación. Asimismo, tomamos un contexto de 40 caracteres para armar las secuencias de entrenamiento X,Y 

In [7]:
val_size_tokens = int(len(encoded) * 0.1)
train_text, val_text = encoded[:-val_size_tokens], encoded[-val_size_tokens:]

CONTEXT_LEN = 40

def build_sequences(array, context_len):
    sequences = [array[i : i+context_len] for i in range(len(array)-context_len)]
    targets   = [array[i+1 : i+context_len+1] for i in range(len(array)-context_len)]
    return np.array(sequences, dtype=np.int16), np.array(targets, dtype=np.int16)

X_train, y_train = build_sequences(train_text, CONTEXT_LEN)
X_val,   y_val   = build_sequences(val_text,   CONTEXT_LEN)
print('Train samples:', X_train.shape[0], '| Val samples:', X_val.shape[0])

Train samples: 2398116 | Val samples: 266421


Train samples: 2398116 | Val samples: 266421

Luego, generamos los Datasets de entrenamiento y validación

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds   = tf.data.Dataset.from_tensor_slices((X_val,   y_val))

BATCH_SIZE = 256
train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

Para poder plantear un early stopping, definiremos un una clase para luego instanciar un callback que nos permita hacerlo. En este caso, por cuestiones de velocidad y simpleza, plantearemos la versión "sencilla y aproximada" de la perplejidad utilizando la exponencial de la cross entropy.

In [9]:
PATIENCE = 4

class PplCallback(keras.callbacks.Callback):
    def __init__(self, patience=PATIENCE):
        super().__init__()
        self.best_ppl = np.inf
        self.wait = 0
        self.patience = patience
    @staticmethod
    def perplexity(loss): return math.exp(loss) if loss < 20 else float('inf')
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        ppl = self.perplexity(val_loss)
        logs['val_perplexity'] = ppl
        print(f'— val_perplexity: {ppl:.3f}')
        if ppl < self.best_ppl:
            self.best_ppl = ppl; self.wait = 0
            self.model.save('best_model.keras')
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print('Early stopping por falta de mejora.')
                self.model.stop_training = True

### Paso 3: definición de modelo, entrenamiento y evaluación

Generamos el modelo seteando inicialmente variables para definir sus parámetros

In [10]:
EMBED_DIM = 128
RNN_UNITS = 256
DROPOUT = 0.1
REC_DROPOUT = 0.1
CLIP_NORM = 1.0

def build_model():
    model = keras.Sequential(name='CharLM_MixedRNN')
    model.add(layers.Input(shape=(None,)))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=EMBED_DIM))
    # Bloque 1: SimpleRNN
    model.add(layers.SimpleRNN(RNN_UNITS, return_sequences=True,
                               dropout=DROPOUT, recurrent_dropout=REC_DROPOUT))
    # Bloque 2: LSTM
    model.add(layers.LSTM(RNN_UNITS, return_sequences=True,
                          dropout=DROPOUT, recurrent_dropout=REC_DROPOUT))
    # Bloque 3: GRU
    model.add(layers.GRU(RNN_UNITS, return_sequences=True,
                         dropout=DROPOUT, recurrent_dropout=REC_DROPOUT))
    # Capa final
    model.add(layers.Dense(vocab_size, activation='softmax'))
    opt = keras.optimizers.RMSprop(learning_rate=1e-3, clipnorm=CLIP_NORM)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt,
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Model: "CharLM_MixedRNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 128)      │         7,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 256)      │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 256)      │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, None, 256)      │       394,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 55)       │        14,135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,039,799 (3.97 MB)

 Trainable params: 1,039,799 (3.97 MB)

 Non-trainable params: 0 (0.00 B)

Para el entrenamiento, planteamos 2 épocas para poder evaluar su evolución debido a que los tiempos de entrenamiento son muy elevados ya que no se cuenta con el hardware necesario para correr este tipo de modelos de forma rápida.

In [11]:
EPOCHS = 2

history = model.fit(
    train_ds, epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[PplCallback()],
)

Epoch 1/2


- Epoch 1/2
  - accuracy: 0.5017
  - loss: 1.6342
  - val_accuracy: 0.5530
  - val_loss: 1.4846
  - val_perplexity: 4.413

- Epoch 2/2
  - accuracy: 0.6350
  - loss: 1.1423
  - val_accuracy: 0.5645
  - val_loss: 1.4558
  - val_perplexity: 4.2880

**Observación:**

Podemos notar que el entrenamiento va mejorando las métricas obtenidas. De contar con mayor capacidad de cómputo es probable que podamos obtener mejores valores para las métricas buscadas.

De los entrenamientos realizados, obtenemos el mejor modelo obtenido de todas las corridas realizadas y guardadas

In [ ]:
best_model = keras.models.load_model('best_model.keras')

#### Paso 4: uso del modelo y análisis de resultados

Para obtener resultados a partir del modelo realizado, debemos definir una manera de elegir el próximo caracter de la secuencia. En ese sentido, definimos dos functions para hacerlo: greedy y beam search

In [ ]:
def encode(text): return pad_sequences([[char2idx[c] for c in text.lower()]], maxlen=CONTEXT_LEN, padding='pre')

def greedy_generate(model, seed, n_chars=200):
    out = seed
    for _ in range(n_chars):
        inp = encode(out)
        pred = model.predict(inp, verbose=0)[0, -1]
        next_char = idx2char[int(np.argmax(pred))]
        out += next_char
    return out

In [ ]:
def decode(indices): return ''.join(idx2char[i] for i in indices)

def beam_search(model, seed, length=200, k=5, temperature=1.0, stochastic=False):
    seed_encoded = encode(seed)[0]
    sequences = [(0.0, list(seed_encoded))]  # (log_prob, seq_tokens)
    for _ in range(length):
        all_candidates = []
        for log_prob, seq in sequences:
            inp = np.array([seq[-CONTEXT_LEN:]])
            probs = model.predict(inp, verbose=0)[0, -1]
            if stochastic:
                probs = softmax(np.log(probs + 1e-10) / temperature)
                idxs = np.random.choice(len(probs), size=k, p=probs, replace=False)
            else:
                idxs = np.argsort(probs)[-k:]
            for idx in idxs:
                candidate = (log_prob + np.log(probs[idx] + 1e-10), seq + [idx])
                all_candidates.append(candidate)
        sequences = sorted(all_candidates, key=lambda tup: tup[0], reverse=True)[:k]
    best_seq = sequences[0][1]
    return decode(best_seq[-(len(seed)+length):])

Luego, realizamos varios ejemplos variando parámetros y seed_text para observar resultados. El código quedará como la última prueba realizada pero se registrarán los resultados obtenidos de pruebas pasadas

In [ ]:
seed_text = "en un lugar "
print('--- Greedy ---')
print(greedy_generate(best_model, seed_text, 400))
print('\n--- Beam search (det) ---')
print(beam_search(best_model, seed_text, 400, k=15))
print('\n--- Beam search (stochastic, T=1.2) ---')
print(beam_search(best_model, seed_text, 400, k=15, temperature=1.2, stochastic=True))

Los resultados obtenidos fueron los siguientes:

**Seed = "Cuando ella llegó" ; length = 30**

- Greedy
  - Cuando ella llegó a la puerta de color con un s

- Beam search (det, k=15)
  - cuando ella llegó a lord henry, dorian gray se 

- Beam search (stochastic, k=15 T=1.6)
  - cuando ella llegó a lord henry, estremeciéndose

**Seed_text = "El sentido de la vida" ; length = 80**

- Greedy 
  - El sentido de la vida se acercó a la puerta de la biblioteca. el retrato se apoderó de la mesa, encon

- Beam search (det, k=15) 
  - el sentido de la vida había abandonado las palabras con una expresión de comprender que el retrato se

- Beam search (stochastic,k=15, T=3) 
  - el sentido de la vida, habían tomado, dorian, dorian, dorian, dorian, dorian, dorian, dorian, quedánd

**Seed_text = "Comer es una forma de" ; length = 160**

- Greedy 
  - Comer es una forma de comprender que el retrato se apoderó de la mesa, encontró allí en la cabeza y se acercó a la puerta de la biblioteca. el retrato se apoderó de la mesa, encontr

- Beam search (det, k=15) 
  - comer es una forma de crueldad en la biblioteca. el retrato se apoderó de la biblioteca. el retrato se apoderó de la biblioteca. el retrato se apoderó de la biblioteca. el retrato s

- Beam search (stochastic, k=15, T=3) 
  - comer es una forma del retrato, y, dorian, dorian, dorian, dorian, dorian, dorian, víctor, habían hecho, dorian, dorian, habían hecho, dorian, dorian, quedándose que, dorian, dorian

**Seed = "en un lugar" y length = 400**
 
- Greedy 
  - en un lugar de la mesa le había abandonado el alba en la cabeza y se acercó a la puerta de la biblioteca. el retrato se apoderó de la mesa, encontró allí en la cabeza y se acercó a la puerta de la biblioteca. el retrato se apoderó de la mesa, encontró allí en la cabeza y se acercó a la puerta de la biblioteca. el retrato se apoderó de la mesa, encontró allí en la cabeza y se acercó a la puerta de la bibliotec

- Beam search (det, k=15) 
  - en un lugar había abandonado las palabras con una expresión de comprender que el retrato se apoderó de la biblioteca. el retrato se detuvo en el retrato. el retrato se le pareció que el retrato se apoderó de la biblioteca. el retrato se le pareció que el retrato se detuvo en el retrato. el retrato se detuvo en el rostro del retrato, estremeciéndose con una expresión de comprender que el retrato se apoderó de 

- Beam search (stochastic,k=15, T=1.2) 
  - en un lugar había abandonado las palabras con una expresión de comprender que el retrato se detuvo en el teatro. el retrato se detuvo en el retrato, estremeciéndose con una expresión que dorian gray había abandonado sin embargo, el retrato se detuvo en el rostro del retrato, estremeciéndose hacia la puerta del retrato, estremeciéndose con una expresión que dorian gray había abandonado sin embargo, el retrato 

**Conclusiones**

A partir de los textos generados, podemos plantear que:

- El modelo logra generar frases gramaticalmente válidas, captando estructuras locales del lenguaje, como concordancia de género, artículos y puntuación.
- Se observan repeticiones y bucles en secuencias largas. Esto ocurre especialmente con greedy search, porque el modelo siempre elige el carácter más probable, pero también se observa en los otros métodos.
- Los resultados del modelo muestran una fuerte tendencia a generar texto relacionado con "El retrato de Dorian Gray", especialmente en las secuencias más largas. Esto se debe a que ese texto, al estar al final del corpus, fue utilizado casi en su totalidad como conjunto de validación. Como consecuencia, el modelo fue optimizado para minimizar la pérdida sobre ese estilo y contenido, generando un sesgo hacia sus patrones narrativos. Para futuros modelos se deberá buscar una forma de mezclar los textos del corpus de forma tal que se evite este sesgo.
- Beam search estocástico con alta temperatura genera texto incoherente y repetitivo, al contrario de lo que a priori uno supondría por la aleatoriedad de su búsqueda.
- El tamaño de contexto limitado (40 caracteres) permite capturar relaciones locales, pero no es suficiente para mantener coherencia temática global. Esto explica por qué el modelo puede arrancar bien pero se desvía o repite más allá de cierto punto.
- El entrenamiento corto (2 épocas) fue suficiente para generalizar frases básicas, pero insuficiente para desarrollar diversidad o profundidad narrativa.